In [7]:
import re
from collections import defaultdict

# Paths to the five text files
file_paths = [
    'Book1.txt',  
    'Book2.txt',  
    'Book3.txt',       
    'Book4.txt',     
    'Book5.txt'          
]

def read_text(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()


THE: 27554
HE : 24462
E P: 2554
 PR: 2524
PRO: 1957
ROJ: 456
OJE: 456
JEC: 745
ECT: 2128
CT : 1070
